In [4]:
using Pkg,LinearAlgebra, SparseArrays, Random,Distributions;

In [5]:
y=[1,2,3]
function get_Y(y)
    m=length(y)#number of rows
    n=m*(m-1) #number of columns
    res = zeros(m,n) #set the result
    start = 1
    for i=1:length(y)
        row = copy(y)
        row = deleteat!(row, i)
        res[i,start:start+m-2]=row
        start = start+m-1
    end
    return res
end
get_Y(y)

3×6 Array{Float64,2}:
 2.0  3.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  3.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  2.0

In [6]:
#fake data
y1=[1,2,3]
y2=[4,5,6]
Y1=get_Y(y1)
Y2=get_Y(y2)
Y=[Y1,Y2]
R=[1 0 0;0 2 0; 0 0 3]
w1=[6,7,8]
w2=[6,3,4]
W=[w1,w2]
t_square = 3
lambda0 = 4
get_Y(y1)

3×6 Array{Float64,2}:
 2.0  3.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  3.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  2.0

In [7]:
function get_para(Y,R,t_square,lambda0)
    m = size(Y[1])[1]#number of rows
    n=m*(m-1) #number of columns
    first = zeros(n,n)
    second=zeros(n,1)
    for i=1:length(Y)
        first += Y[i]'*inv(R)*Y[i]
        second += Y[i]'*inv(R)*W[i]
    end
    first += Diagonal(repeat([t_square],n))
    second += repeat([lambda0*t_square],n)
    mu = vec(inv(first)*second)
    return mu,Symmetric(inv(first))
end


#MCMC
mu,var = get_para(Y,R,t_square,lambda0)
rand(MvNormal(mu,Symmetric(var)))

6-element Array{Float64,1}:
 1.1853591066798517
 0.5595683473769499
 1.2850327876214134
 0.9013032139006962
 1.5191068511665973
 0.8959346971400703

### My case

In [8]:
y = [1,2,3]
function get_my_Y(y)
    m=length(y)#number of rows
    n::Int64=(m*(m-1))/2 #number of columns
    res = zeros(m,n) #set the result
    start = 1
    for i = 1:length(y)-1
        row = y[2:i+1]
        res[i+1,start:start + i - 1] = row
        start = start + i
    end
    return res
end
get_my_Y(y)

3×3 Array{Float64,2}:
 0.0  0.0  0.0
 2.0  0.0  0.0
 0.0  2.0  3.0

In [9]:
#Making fake data
y1=[1,2,3]
y2=[4,5,6]
Y1=get_my_Y(y1)
Y2=get_my_Y(y2)
Y=[Y1,Y2]
R=[1 0 0;0 2 0; 0 0 3]
w1=[6,7,8]
w2=[6,3,4]
W=[w1,w2]
t_square = 3
lambda0 = 4

4

In [10]:
function get_para(Y,R,t_square,lambda0,W)
    m = size(Y[1])[1]#number of rows
    n::Int64=(m*(m-1))/2 #number of columns
    first = zeros(n,n)
    second=zeros(n,1)
    for i=1:length(Y)
        first += Y[i]'*inv(R)*Y[i]
        second += Y[i]'*inv(R)*W[i]
    end
    first += Diagonal(repeat([t_square],n))
    second += repeat([lambda0*t_square],n)    
    mu = vec(inv(first)*second)
    return mu,Symmetric(inv(first))
end

@time mu,var = get_para(Y,R,t_square,lambda0,W)
#rand(MvNormal(mu,Symmetric(var)))

  0.062743 seconds (66.29 k allocations: 3.308 MiB)


([1.51429, 1.14286, 0.793651], [0.0571429 -0.0 -0.0; -0.0 0.214286 -0.142857; -0.0 -0.142857 0.150794])

In [11]:
mu

3-element Array{Float64,1}:
 1.5142857142857142
 1.1428571428571435
 0.7936507936507936

In [12]:
var

3×3 Symmetric{Float64,Array{Float64,2}}:
  0.0571429  -0.0       -0.0     
 -0.0         0.214286  -0.142857
 -0.0        -0.142857   0.150794

## Sampling by cholesky decomposition

In [936]:
res = randn(3)
res = L.U*res + mu
function sampling(mu,var,n)
    res = randn(n)
    L = cholesky(var)
    res = L.L*res + mu
    return res
end
sampling(mu,var,3)

3-element Array{Float64,1}:
 1.3077669864914216 
 1.9542904314155036 
 0.18654086366161748

In [13]:
#Making fake data
y1=[1,2,3]
y2=[4,5,6]
Y1=get_my_Y(y1)
Y2=get_my_Y(y2)
Y=[Y1,Y2]
R=[1 0 0;0 2 0; 0 0 3]
w1=[6 7 8]
w2=[6 3 4]
W=[w1 w2]
t_square = 3
lambda0 = 4
number=2


2

In [15]:
#Y is the data for all individuals

#unlist function will tranfrom y from a list to a big matrix
function unlist(Y)
    res = Y[1]
    for i = 2:length(Y)
        res = vcat(res,Y[i])
    end
    return res
end

unlist(Y)

6×3 Array{Float64,2}:
 0.0  0.0  0.0
 2.0  0.0  0.0
 0.0  2.0  3.0
 0.0  0.0  0.0
 5.0  0.0  0.0
 0.0  5.0  6.0

In [18]:
function get_para(Y,R,W,t_square,lambda0,number)
    m = size(Y[1])[1]#number of rows
    n::Int64=(m*(m-1))/2 #number of columns
    Y = unlist(Y) #Set big Y matrix
    
    #Define sparse matrix
    I = collect(1:number*m); J=collect(1:number*m);V = repeat(diag(R),number)
    R = sparse(I,J,V)
    
    #formula calculation
    first = Y'*inv(Matrix(R))*Y + Diagonal(ones(m,m))
    second = Y'*inv(Matrix(R))*transpose(vcat(W)) + ones(m,1)*lambda0*t_square
    mu = vec(inv(first)*second)
    return mu,Symmetric(inv(first))
end



get_para (generic function with 3 methods)

In [19]:
a, b = get_para(Y,R,W,t_square,lambda0,number)

([1.70968, 1.8, 0.4], [0.0645161 0.0 -0.0; 0.0 0.6 -0.45; -0.0 -0.45 0.4])

### Sparse Version

In [20]:
#Y is a data for single persion
y1 = [1,2,3]
function get_sparse_Y(Y)
    m = size(Y)[1] #number of rows
    n::Int64=(m*(m-1))/2  #number of columns
    col_index = collect(1:n)
    row_index = [];value = []
    for i = 2:m
        current = fill(i,i-1) #get a element i with i-1 time
        row_index = vcat(row_index,current)
        current_value = Y[2:i]
        value = vcat(value,current_value)
    end
    
    #Return the sparse matrix
    row_index = convert(Array{Int64,1}, row_index) #change Any type
    value = convert(Array{Float64,1}, value) #Change Any type

    res = sparse(row_index,col_index,value)
end

get_sparse_Y(y1)

3×3 SparseMatrixCSC{Float64,Int64} with 3 stored entries:
  [2, 1]  =  2.0
  [3, 2]  =  2.0
  [3, 3]  =  3.0

In [21]:
#Fake data
y1=[1,2,3]
y2=[4,5,6]
ys1=get_sparse_Y(y1)
ys2=get_sparse_Y(y2)
YS = [ys1,ys2]
R=[1 0 0;0 2 0; 0 0 3]
w1=[6,7,8]
w2=[6,3,4]
W=[w1,w2]
t_square = 3
lambda0 = 4
m = 3
number = 10000

10000

In [22]:
YS = repeat([get_sparse_Y(rand(3))],10000)
YS = unlist(YS)
#W=unlist(W)
W = rand(30000)

function get_sparse_para(YS,R,W,t_square,lambda0,number,m)
    
    #Define residual matrix
    value = diag(R).^(-1)
    RS = kron(sparse(I,number,number),sparse(1:m,1:m,value))
    
    #formula calculation
    YS_RS_product = YS'*RS
    first = YS_RS_product*YS + t_square*sparse(I,m,m)
    second = YS_RS_product*W + ones(m,1)*lambda0*t_square
    
    #Return the result
    first = inv(Matrix(first))
    mu = vec(first*second)
    var = Symmetric(first)
    
    #Take sample
    res = rand(MvNormal(mu,var))
    return res
end

get_sparse_para (generic function with 1 method)

In [23]:
#a,b = get_sparse_para(YS,R,W,t_square,lambda0,number,3)#
@time lambda= get_sparse_para(YS,R,W,t_square,lambda0,number,m)

  0.946754 seconds (1.84 M allocations: 93.388 MiB, 3.52% gc time)


3-element Array{Float64,1}:
 1.729572210267744 
 1.2724447603244464
 0.3541255587180224

In [24]:
y1 = rand(1000);y2 = rand(1000);y3=rand(1000)
BigY = [y1;y2;y3]
Lambda = [1 0 0; lambda[1] 1 0;lambda[2] lambda[3] 1]

function Modify_Y(Lambda,Y,n_individuals)
    Identy = Matrix{Int32}(I,n_individuals,n_individuals)
    res = kron(Identy,Lambda)*BigY
    return res
end

Modify_Y(Lambda,BigY,1000)

3000-element Array{Float64,1}:
 0.34802452134817874
 1.4149432317713164 
 1.0652942817884665 
 0.7825825792166521 
 1.4453350020348745 
 1.7537673738276358 
 0.4599740409604103 
 1.2862978406737835 
 1.338930364090148  
 0.9104190191302641 
 1.6104364391544777 
 1.4144511002748166 
 0.09554676517904537
 ⋮                  
 0.8620076163038042 
 2.471519559559001  
 1.5471924570753162 
 0.9266985378004544 
 1.7095096473746705 
 1.2343259397796176 
 0.4676316011375379 
 0.8810097785854595 
 1.3904296339995867 
 0.6434511729910133 
 1.5702555697471807 
 1.5532405167262113 